In [1]:
import json
import csv
import time
from datetime import datetime, timedelta
from numpy import array
import numpy as np
import pandas as pd

In [2]:
symbols = ["OTO", "HPT", "AMANAH"]
items = ["2017-02-14", "2017-02-15", "2017-02-16", "2017-02-17"]
# items = ["2017-02-28"]

In [3]:
def getFreefloat():
    allShare = 250000000  # ACAP
    freeFloat = 67.03
    return (allShare * freeFloat)/100

def check(list):
    if list[1] is not 0 and list[2] is not 0 and list[4] is 0: return 0
    else: return 1

def getSumVol(list):
    # Sum = list[0] + list[1] + list[2] + list[3] + list[4]
    Sum = np.sum(list)
    return Sum

def event(eve):
    if eve == "B":
        return 1
    elif eve == "S":
        return 0
    else:
        return 5

def getTime(time):
    timeSplit = time.split(':')
    return timeSplit

def getDate(Date):
    dateSplit = Date.split('-')
    return dateSplit


def Timestamp2Datetime(Timestamp):
    Datetime = datetime.fromtimestamp(Timestamp-25200).strftime('%Y-%m-%d-%H-%M-%S')
    # print Datetime
    return Datetime

def Datetime2Timestamp(dt, epoch=datetime(1970,1,1)):
    td = dt - epoch
    # return td.total_seconds()
    return (td.microseconds + (td.seconds + td.days * 86400) * 10**6) // 10**6


def toTemp(price, vol, eve, symbol):
    sumVO = getSumVol(vol)
    compareBid1[symbol] = tempBid[symbol][5]
    compareOff1[symbol] = tempOffer[symbol][5]


    if eve == 1:

        tempBid[symbol] = price + vol

        difBO[symbol] = sumVO - tempBidVol[symbol]

        if difBO[symbol] <= -1000000:
            volBO[symbol] = -4
        elif -1000000 < difBO[symbol] <= -500000:
            volBO[symbol] = -3
        elif -500000 < difBO[symbol] <= -100000:
            volBO[symbol] = -2
        elif -100000 < difBO[symbol] <= -100:
            volBO[symbol] = -1
        elif -100 < difBO[symbol] <= 100000:
            volBO[symbol] = 1
        elif 100000 < difBO[symbol] <= 500000:
            volBO[symbol] = 2
        elif 500000 < difBO[symbol] <= -1000000:
            volBO[symbol] = 3
        elif difBO[symbol] > 1000000:
            volBO[symbol] = 4

        tempBidVol[symbol] = sumVO
        return price + vol + tempOffer[symbol] + [tempBidVol[symbol]] + [tempOffVol[symbol]]
    else:

        tempOffer[symbol] = price + vol

        difBO[symbol] = sumVO - tempOffVol[symbol]

        if difBO[symbol] <= -1000000:
            volBO[symbol] = -40
        elif -1000000 < difBO[symbol] <= -500000:
            volBO[symbol] = -30
        elif -500000 < difBO[symbol] <= -100000:
            volBO[symbol] = -20
        elif -100000 < difBO[symbol] <= -100:
            volBO[symbol] = -10
        elif -100 < difBO[symbol] <= 100000:
            volBO[symbol] = 10
        elif 100000 < difBO[symbol] <= 500000:
            volBO[symbol] = 20
        elif 500000 < difBO[symbol] <= -1000000:
            volBO[symbol] = 30
        elif difBO[symbol] > 1000000:
            volBO[symbol] = 40

        tempOffVol[symbol] = sumVO
        return tempBid[symbol] + price + vol + [tempBidVol[symbol]] + [tempOffVol[symbol]]



def checkTimestamp(timestamp,symbol):
    if timestamp < tempTimestamp[symbol]:
        return tempTimestamp[symbol],1
    else:
        tempTimestamp[symbol] = timestamp
        return timestamp,0










def getSpread(x):
    ''' x<2         0.01    0
        2<=x<5       0.02    1
        5<=x<10      0.05    2
        10<=x<25     0.1     3
        25<=x<100    0.25    4
        100<=x<200   0.5     5
        200<=x<400   1.0     6
        x>=400       2.0     7
    '''
    if x < 200:
        return 0, 1, 200
    elif 200 <= x < 500:
        return 1, .5, 500
    elif 500 <= x < 1000:
        return 2, .2, 1000
    elif 1000 <= x < 2500:
        return 3, .1, 2500
    elif 2500 <= x < 10000:
        return 4, .04, 10000
    elif 10000 <= x < 20000:
        return 5, .02, 20000
    elif 20000 <= x < 40000:
        return 6, .01, 40000
    else:
        return 7, .005, 40000

def getDifSpread(x1, x2):
    a1, t1, z1 = getSpread(x1)
    a2, t2, z2 = getSpread(x2)
    if a1 == a2:
        Dif = (x2 - x1) * t1
    elif a2 > a1:
        Dif = ((x2 - z1) * t2) + ((z1 - x1) * t1)
    else:
        Dif = -(((x1 - z2) * t1) + ((z2 - x2) * t2))
    return int(Dif)



def calmoney(pbid0,pbid1,pbid2,pbid3,pbid4,vbid0,vbid1,vbid2,vbid3,vbid4):
    money = ((pbid0*vbid0)+(pbid1*vbid1)+(pbid2*vbid2)+(pbid3*vbid3)+(pbid4*vbid4))/100
    return money

In [4]:
idSymbol = dict()
tempBid = dict()
tempOffer = dict()
tempBidVol = dict()
tempOffVol = dict()
tempAll = dict()
priATO = dict()
volATO = dict()
tempTimestamp = dict()
marketStatus = dict()
forTemp = dict()
clearNoise = dict()

sumOrder = dict()
countOrder = dict()

lastPrice = dict()
prior = dict()
highPrice = dict()
lowPrice = dict()
avgPrice = dict()
tradeVol = dict()
buyVol = dict()
sellVol = dict()
auctVol = dict()

spread = dict()
count100k = dict()
count1m = dict()
difBO = dict()

tf30 = dict()
volBO = dict()
typeBO = dict()
compareBid1 = dict()
compareOff1 = dict()


allbidmoney = dict()
sidetransac = dict()

dfall = dict()



stock =dict()
stock1 =dict()
ato = dict()

for symbol in symbols:
    # tempBid[symbol] = [None, None, None, None, None, None, None, None, None, None]
    # tempOffer[symbol] = [None, None, None, None, None, None, None, None, None, None]
    tempBid[symbol] = [0,0,0,0,0,0,0,0,0,0]
    tempOffer[symbol] = [0,0,0,0,0,0,0,0,0,0]
    # forTemp[symbol] = [None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None]

    priATO[symbol] = 0
    volATO[symbol] = 0
    tempTimestamp[symbol] = 0
    marketStatus[symbol] = 0
    clearNoise[symbol] = 0
    idSymbol[symbol] = 0

    sumOrder[symbol] = 0
    countOrder[symbol] = 0

    lastPrice[symbol] = 0
    prior[symbol] = 0
    highPrice[symbol] = 0
    lowPrice[symbol] = 0
    avgPrice[symbol] = 0
    tradeVol[symbol] = 0
    buyVol[symbol] = 0
    sellVol[symbol] = 0
    auctVol[symbol] = 0

    spread[symbol] = 0
    count100k[symbol] = 0
    count1m[symbol] = 0
    difBO[symbol] =0
    tf30[symbol] = 0
    volBO[symbol] = 0
    typeBO[symbol] = 0

    tempBidVol[symbol] = 0
    tempOffVol[symbol] = 0
    compareBid1[symbol] = 0
    compareOff1[symbol] = 0

    allbidmoney[symbol] = 0
    sidetransac[symbol] = 0
    
    stock[symbol] = pd.DataFrame()
    stock1[symbol] = pd.DataFrame()
    ato[symbol] = pd.DataFrame()
    


for symbol in symbols:
    forTemp[symbol] = []

In [6]:
t0 = time.time()

# ref_files = [open("TESTIPython\\List\\" + Symbol + ".csv", "a") for Symbol in symbols]
# ref_files2 = [open("TESTIPython\\Listato\\" + Symbol + ".csv", "a") for Symbol in symbols]



three_tuple = [(current, items[idx - 1] if idx >= 1 else None, items[idx + 1] if idx < len(items) - 1 else None) for idx, current in enumerate(items)]
# return a 3-tuple with current, previous and next elements

    

for enum, Date in enumerate(items):

    yesterday = three_tuple[enum][1]
    today = three_tuple[enum][0]
    tomorrow = three_tuple[enum][2]
    
    YYYY,MM,DD = getDate(Date)

    YYYY = int(YYYY)
    MM = int(MM)
    DD = int(DD)
    
    with open("INPUT\\" +Date+".dat") as f:

        content = f.readlines()
        for line in content:
            if 'data' in line:
                try:
                    jsonData = line.replace("data: ", "")
                    jsonDecoded = json.loads(jsonData)
                except ValueError:
                    # print Date, jsonData
                    continue

                if 'sym' in jsonDecoded.keys():
                    sym = jsonDecoded["sym"]
                    if sym in symbols:
                        if 'isf' in jsonDecoded.keys():

                            pri = jsonDecoded["pri"]  # price
                            priATO[sym] = pri

                            tim = jsonDecoded["tim"]    #time
                            HH, mm, SS = getTime(jsonDecoded["tim"])            ###
                            HH = int(HH)
                            mm = int(mm)
                            SS = int(SS)
                            Timestamp = Datetime2Timestamp(datetime(YYYY, MM, DD, HH, mm, SS))  ###
                            Timestamp = checkTimestamp(Timestamp,sym)

                            vol = jsonDecoded["vol"]  # volume
                            volATO[sym] = vol

                            op1 = jsonDecoded["op1"]  # open1
                            op2 = jsonDecoded["op2"]  # open2

                            isf = jsonDecoded["isf"]
                            if isf == 'F':
                                marketStatus[sym] = 0   #Pre-Open1
                                if HH == 14:
                                    marketStatus[sym] = 2   #Pre-Open2
                                if HH == 16:
                                    marketStatus[sym] = 3  # Pre-Open2

                            elif isf == 'T':
                                marketStatus[sym] = 1   #Open
                                clearNoise[sym] = Timestamp[0]

                                if HH == 16 and mm >30:
                                    marketStatus[sym] = 4   #Pre-Close
                            else:
                                marketStatus[sym] = 5
                                print "Unknown status"



#                                 table = [marketStatus[sym]] + [Timestamp[0]] + [Date] + [tim] + [pri] + [vol] + [op1] + [op2]
#                                 df2 = pd.DataFrame(table)
#                                 ato[sym] = ato[sym].append(df2, ignore_index=True)

                            ato[sym] = ato[sym].append({'marketStatus':marketStatus[sym], 'Timestamp':Timestamp[0], 'Date':Date, 'time':tim, 'pri':pri, 'vol':vol, 'op1':op1, 'op2':op2}, ignore_index=True)

                        elif 'time' in jsonDecoded.keys():
                            pri = jsonDecoded["pri"]
                            if check(pri) is 0:
                                continue

                            eve = event(jsonDecoded["sid"])

                            tim = jsonDecoded["time"]
                            HH, mm, SS = getTime(jsonDecoded["time"])  ###
                            HH = int(HH)
                            mm = int(mm)
                            SS = int(SS)

                            Timestamp = Datetime2Timestamp(datetime(YYYY, MM, DD, HH, mm, SS))  ###
                            Timestamp = checkTimestamp(Timestamp,sym)

                            tempY, tempM, tempD, HH, mm, SS = getDate(Timestamp2Datetime(Timestamp[0]))
                            HH = int(HH)
                            mm = int(mm)
                            SS = int(SS)



                            ids = jsonDecoded["id"]  ##identify number of stock
                            idSymbol[sym] = ids

                            vol = jsonDecoded["vol"]

                            # if jsonDecoded["sid"] == 'S':
                            #     side = 1  # Fill Offer
                            # elif jsonDecoded["sid"] == 'B':
                            #     side = 0  # Fill Bid
                            # else:
                            #     side = 3

                            side = event(jsonDecoded["sid"])

                            bidOffer = toTemp(pri, vol, eve, sym)


                            allbidmoney[sym] = calmoney(bidOffer[0],bidOffer[1],bidOffer[2],bidOffer[3],bidOffer[4],bidOffer[5],bidOffer[6],bidOffer[7],bidOffer[8],bidOffer[9])




                            if bidOffer[0] == 0 and marketStatus[sym] != 1:
                                bidOffer[0] = priATO[sym]
                            if bidOffer[10] == 0 and marketStatus[sym] != 1:
                                bidOffer[10] = priATO[sym]

                            if marketStatus[sym] == 1:
                                spread[sym] = getDifSpread(prior[sym],bidOffer[0])

                                if HH < 10:
                                    tf30[sym] = 0
                                elif HH == 10 and mm < 30:
                                    tf30[sym] = 1
                                elif HH == 10 and mm >= 30:
                                    tf30[sym] = 2
                                elif HH == 11 and mm < 30:
                                    tf30[sym] = 3
                                elif HH == 11 and mm >= 30:
                                    tf30[sym] = 4
                                elif HH == 12 and mm <= 30:
                                    tf30[sym] = 5
                                elif HH == 14 and mm < 30:
                                    tf30[sym] = 6
                                elif HH == 14 and mm >= 30:
                                    tf30[sym] = 7
                                elif HH == 15 and mm < 30:
                                    tf30[sym] = 8
                                elif HH == 15 and mm >= 30:
                                    tf30[sym] = 9
                                elif HH == 16 and mm <= 30:
                                    tf30[sym] = 10


                            # if side == 1:   #B(Bid)=1, S(Offer)=0
                            #     if sumOrder[sym] == 0:
                            #         if bidOffer[5] != tempBid[sym][5]:
                            #             typeBO = 0
                            #         else:
                            #
                            # if bidOffer[15] != tempOffer[sym][5]:
                            if sumOrder[sym] != 0:
                                volBO[sym] = sumOrder[sym]
                                # typeBO[sym] = side
                                if sidetransac[sym] == 0:
                                    typeBO[sym] = 'Sell'
                                else:
                                    typeBO[sym] = 'Buy'
                            else:
                                if side == 1:
                                    if compareBid1[sym] == bidOffer[5]:
                                        typeBO[sym] = 'Bid Not slot 0'
                                    else:
                                        typeBO[sym] = 'Bid is slot 0'
                                else:
                                    if compareOff1[sym] == bidOffer[15]:
                                        typeBO[sym] = 'Offer Not slot 0'
                                    else:
                                        typeBO[sym] = 'Offer is slot 0'

                            # a = [marketStatus[sym]] + [id] + [Timestamp[0]]+ [Timestamp[1]] + [side]+ [Date] + [tim] + bidOffer

                            # forTemp[sym] = [Date] + [tim] + [idSymbol[sym]] + [marketStatus[sym]] + [Timestamp[0]]+ [Timestamp[1]] + [side] + bidOffer + \
                            #                [volATO[sym]] + [sumOrder[sym]] + [countOrder[sym]] + \
                            #                [tradeVol[sym]] + [lastPrice[sym]] + [prior[sym]] + [highPrice[sym]] + [lowPrice[sym]] + [avgPrice[sym]] + \
                            #                [gain[sym]] + [count100k[sym]] + [buyVol[sym]] + [sellVol[sym]] + [auctVol[sym]]



#                                 forTemp[sym] = [tf30[sym]] + [Date] + [tim] + [HH] + [mm] + [SS] + [idSymbol[sym]]  + [Timestamp[0]] + [Timestamp[1]] + bidOffer + \
#                                                [difBO[sym]] + [volBO[sym]] + [typeBO[sym]] + [allbidmoney[sym]] +[compareOff1[sym]] + [bidOffer[15]] + [side] + [sumOrder[sym]] + [countOrder[sym]] + \
#                                                [tradeVol[sym]] + [lastPrice[sym]] + [prior[sym]] + [highPrice[sym]] + [lowPrice[sym]] + [avgPrice[sym]] + \
#                                                [spread[sym]] + [count100k[sym]] + [buyVol[sym]] + [sellVol[sym]] + [auctVol[sym]]


                            if clearNoise[sym] == Timestamp[0]:
                                continue

                            if HH == 9 and mm == 30:
                                continue
                            # if marketStatus[sym] == 4:
                            if marketStatus[sym] != 1:  ##selective row when market opened only.
                                continue

#                                     wr = csv.writer(ref_files2[symbols.index(sym)], lineterminator='\n')
#                                     wr.writerow(forTemp[sym])

                                # np.array(forTemp[sym]).dump(open('array2.npy', 'wb'))

                            else:
#                                     wr = csv.writer(ref_files[symbols.index(sym)], lineterminator='\n')
#                                     wr.writerow(forTemp[sym])
                                stock[sym] = stock[sym].append({'tf30':tf30[sym], 'Timestamp':Timestamp[0], 'Date':Date, 'yesterday':yesterday, 'today': today, 'tomorrow':tomorrow, 'time':tim, 'BidOffer':bidOffer, 'difBO':difBO[sym], 'typeBO':typeBO[sym], 'spread':spread[sym]}, ignore_index=True)


                                # sql = "INSERT INTO %s (hour,minute,second,timestamp,bid0,bid1,bid2,bid3,bid4) VALUES ('%s','%s','%s','%s','%s','%s','%s','%s','%s')" % (
                                #     sym, HH, mm, SS, Timestamp[0], bidOffer[0], bidOffer[1], bidOffer[2],
                                #     bidOffer[3], bidOffer[4])
                                #
                                # try:
                                #     cursor.execute(sql)
                                #     # db.commit()
                                # except Exception as e:
                                #     print e
                                #     db.rollback()

                                # np.array(forTemp[sym]).dump(open('array.npy', 'wb'))

                                sumOrder[sym] = 0
                                countOrder[sym] = 0

                        elif 'ava' in jsonDecoded.keys():
                            tim = jsonDecoded["tim"]  ##time
                            HH, mm, SS = getTime(jsonDecoded["tim"])
                            HH = int(HH)
                            mm = int(mm)
                            SS = int(SS)
                            Timestamp = Datetime2Timestamp(datetime(YYYY, MM, DD, HH, mm, SS))  ###
                            Timestamp = checkTimestamp(Timestamp,sym)

                            actVol = jsonDecoded["vol"]  ##vol action
                            prc = jsonDecoded["prc"]  # Last price action(Baht) must / 100
                            lastPrice[sym] = prc

                            # ava = jsonDecoded["ava"]  # Total Trade Value(Baht) must / 100
                            avo = jsonDecoded["avo"]  # Total Trade Volume(Share)
                            tradeVol[sym] = avo

                            bvo = jsonDecoded["bvo"]  # Net Buy Volume(Share)
                            svo = jsonDecoded["svo"]  # Net Sell Volume(Share)
                            ovo = jsonDecoded["ovo"]  # Net ATO/ATC Volume(Share)

                            buyVol[sym] = bvo
                            sellVol[sym] = svo
                            auctVol[sym] = ovo

                            # sid = jsonDecoded["sid"]  ## B = Buy, S = Sell

                            prr = jsonDecoded["prr"]  # Previous Close must / 100
                            hgh = jsonDecoded["hgh"]  # high Price must / 100
                            low = jsonDecoded["low"]  # low Price must / 100
                            avg = jsonDecoded["avg"]  # Average Price must / 100
                            prior[sym] = prr
                            highPrice[sym] = hgh
                            lowPrice[sym] = low
                            avgPrice[sym] = avg

                            sidetransac[sym] = event(jsonDecoded["sid"])

                            sumOrder[sym] += actVol
                            countOrder[sym] += 1

                            if sumOrder[sym] >= 100000:
                                count100k[sym] = 1
                            elif sumOrder[sym] >= 1000000:
                                count1m[sym] = 1    
                            else:
                                count100k[sym] = 0
                                count1m[sym] = 0
                            
                            

                                
t1 = time.time()
total = t1-t0

print total

661.256000042


In [8]:
stock['HPT']

,BidOffer,Date,Timestamp,difBO,spread,tf30,time,today,tomorrow,typeBO,yesterday
0,"[130, 129, 128, 127, 126, 59800, 134100, 26700...",2017-02-14,1.487066e+09,20000.0,130.0,0.0,09:57:03,2017-02-14,2017-02-15,Offer Not slot 0,None
1,"[130, 129, 128, 127, 126, 109800, 134100, 2670...",2017-02-14,1.487066e+09,50000.0,130.0,1.0,10:00:20,2017-02-14,2017-02-15,Bid is slot 0,None
2,"[130, 129, 128, 127, 126, 109800, 134100, 2672...",2017-02-14,1.487067e+09,200.0,130.0,1.0,10:02:29,2017-02-14,2017-02-15,Bid Not slot 0,None
3,"[130, 129, 128, 127, 126, 109800, 134100, 2672...",2017-02-14,1.487067e+09,100000.0,130.0,1.0,10:02:40,2017-02-14,2017-02-15,Bid Not slot 0,None
4,"[130, 129, 128, 127, 126, 109800, 134100, 2672...",2017-02-14,1.487067e+09,10000.0,130.0,1.0,10:02:55,2017-02-14,2017-02-15,Offer Not slot 0,None
5,"[130, 129, 128, 127, 126, 109800, 134100, 2672...",2017-02-14,1.487067e+09,10000.0,130.0,1.0,10:03:10,2017-02-14,2017-02-15,Offer Not slot 0,None
6,"[130, 129, 128, 127, 126, 109800, 134100, 2672...",2017-02-14,1.487067e+09,10000.0,130.0,1.0,10:03:20,2017-02-14,2017-02-15,Bid Not slot 0,None
7,"[130, 129, 128, 127, 126, 109800, 134100, 2672...",2017-02-14,1.487067e+09,10000.0,130.0,1.0,10:03:20,2017-02-14,2017-02-15,Offer Not slot 0,None
8,"[130, 129, 128, 127, 126, 50800, 134100, 26720...",2017-02-14,1.487067e+09,-59000.0,0.0,1.0,10:03:22,2017-02-14,2017-02-15,Sell,None
9,"[129, 128, 127, 126, 125, 134100, 267200, 1780...",2017-02-14,1.487067e+09,120200.0,-1.0,1.0,09:56:11,2017-02-14,2017-02-15,Sell,None


In [54]:
## confused, why dataframes are Null value ##
# t0 = time.time()

# ref_files = [open("TESTIPython\\List\\" + Symbol + ".csv", "a") for Symbol in symbols]
# ref_files2 = [open("TESTIPython\\Listato\\" + Symbol + ".csv", "a") for Symbol in symbols]


# for Date in items:

#     YYYY,MM,DD = getDate(Date)

#     YYYY = int(YYYY)
#     MM = int(MM)
#     DD = int(DD)
    
#     with open("INPUT\\" +Date+".dat") as f:
            
#         content = f.readlines()
#         for line in content:
#             if 'data' in line:
#                 try:
#                     jsonData = line.replace("data: ", "")
#                     jsonDecoded = json.loads(jsonData)
#                 except ValueError:
#                     # print Date, jsonData
#                     continue

#                 if 'sym' in jsonDecoded.keys():
#                     sym = jsonDecoded["sym"]

#                     if sym in symbols:
#                         if 'time' in jsonDecoded.keys():
#                             pri = jsonDecoded["pri"]
#                             if check(pri) is 0:
#                                 continue

#                             eve = event(jsonDecoded["sid"])

#                             tim = jsonDecoded["time"]
#                             HH, mm, SS = getTime(jsonDecoded["time"]) 
#                             HH = int(HH)
#                             mm = int(mm)
#                             SS = int(SS)

#                             Timestamp = Datetime2Timestamp(datetime(YYYY, MM, DD, HH, mm, SS))  ###
#                             Timestamp = checkTimestamp(Timestamp,sym)

#                             tempY, tempM, tempD, HH, mm, SS = getDate(Timestamp2Datetime(Timestamp[0]))
#                             HH = int(HH)
#                             mm = int(mm)
#                             SS = int(SS)



#                             ids = jsonDecoded["id"]  ##identify number of stock
#                             idSymbol[sym] = ids

#                             vol = jsonDecoded["vol"]

#                             # if jsonDecoded["sid"] == 'S':
#                             #     side = 1  # Fill Offer
#                             # elif jsonDecoded["sid"] == 'B':
#                             #     side = 0  # Fill Bid
#                             # else:
#                             #     side = 3

#                             side = event(jsonDecoded["sid"])

#                             bidOffer = toTemp(pri, vol, eve, sym)


#                             allbidmoney[sym] = calmoney(bidOffer[0],bidOffer[1],bidOffer[2],bidOffer[3],bidOffer[4],bidOffer[5],bidOffer[6],bidOffer[7],bidOffer[8],bidOffer[9])




#                             if bidOffer[0] == 0 and marketStatus[sym] != 1:
#                                 bidOffer[0] = priATO[sym]
#                             if bidOffer[10] == 0 and marketStatus[sym] != 1:
#                                 bidOffer[10] = priATO[sym]

#                             if marketStatus[sym] == 1:
#                                 spread[sym] = getDifSpread(prior[sym],bidOffer[0])

#                                 if HH < 10:
#                                     tf30[sym] = 0
#                                 elif HH == 10 and mm < 30:
#                                     tf30[sym] = 1
#                                 elif HH == 10 and mm >= 30:
#                                     tf30[sym] = 2
#                                 elif HH == 11 and mm < 30:
#                                     tf30[sym] = 3
#                                 elif HH == 11 and mm >= 30:
#                                     tf30[sym] = 4
#                                 elif HH == 12 and mm <= 30:
#                                     tf30[sym] = 5
#                                 elif HH == 14 and mm < 30:
#                                     tf30[sym] = 6
#                                 elif HH == 14 and mm >= 30:
#                                     tf30[sym] = 7
#                                 elif HH == 15 and mm < 30:
#                                     tf30[sym] = 8
#                                 elif HH == 15 and mm >= 30:
#                                     tf30[sym] = 9
#                                 elif HH == 16 and mm <= 30:
#                                     tf30[sym] = 10


#                             # if side == 1:   #B(Bid)=1, S(Offer)=0
#                             #     if sumOrder[sym] == 0:
#                             #         if bidOffer[5] != tempBid[sym][5]:
#                             #             typeBO = 0
#                             #         else:
#                             #
#                             # if bidOffer[15] != tempOffer[sym][5]:
#                             if sumOrder[sym] != 0:
#                                 volBO[sym] = sumOrder[sym]
#                                 # typeBO[sym] = side
#                                 if sidetransac[sym] == 0:
#                                     typeBO[sym] = 'Sell'
#                                 else:
#                                     typeBO[sym] = 'Buy'
#                             else:
#                                 if side == 1:
#                                     if compareBid1[sym] == bidOffer[5]:
#                                         typeBO[sym] = 'Bid Not slot 0'
#                                     else:
#                                         typeBO[sym] = 'Bid is slot 0'
#                                 else:
#                                     if compareOff1[sym] == bidOffer[15]:
#                                         typeBO[sym] = 'Offer Not slot 0'
#                                     else:
#                                         typeBO[sym] = 'Offer is slot 0'

#                             # a = [marketStatus[sym]] + [id] + [Timestamp[0]]+ [Timestamp[1]] + [side]+ [Date] + [tim] + bidOffer

#                             # forTemp[sym] = [Date] + [tim] + [idSymbol[sym]] + [marketStatus[sym]] + [Timestamp[0]]+ [Timestamp[1]] + [side] + bidOffer + \
#                             #                [volATO[sym]] + [sumOrder[sym]] + [countOrder[sym]] + \
#                             #                [tradeVol[sym]] + [lastPrice[sym]] + [prior[sym]] + [highPrice[sym]] + [lowPrice[sym]] + [avgPrice[sym]] + \
#                             #                [gain[sym]] + [count100k[sym]] + [buyVol[sym]] + [sellVol[sym]] + [auctVol[sym]]



#                             forTemp[sym] = [tf30[sym]] + [Date] + [tim] + [HH] + [mm] + [SS] + [idSymbol[sym]]  + [Timestamp[0]] + [Timestamp[1]] + bidOffer + \
#                                            [difBO[sym]] + [volBO[sym]] + [typeBO[sym]] + [allbidmoney[sym]] +[compareOff1[sym]] + [bidOffer[15]] + [side] + [sumOrder[sym]] + [countOrder[sym]] + \
#                                            [tradeVol[sym]] + [lastPrice[sym]] + [prior[sym]] + [highPrice[sym]] + [lowPrice[sym]] + [avgPrice[sym]] + \
#                                            [spread[sym]] + [count100k[sym]] + [buyVol[sym]] + [sellVol[sym]] + [auctVol[sym]]


#                             if clearNoise[sym] == Timestamp[0]:
#                                 continue

#                             if HH == 9 and mm == 30:
#                                 continue
#                             # if marketStatus[sym] == 4:
#                             if marketStatus[sym] != 1:  ##selective row when market opened only.
#                                 continue
# #                                 wr = csv.writer(ref_files2[symbols.index(sym)], lineterminator='\n')
# #                                 wr.writerow(forTemp[sym])

# #                                 np.array(forTemp[sym]).dump(open('array2.npy', 'wb'))
#                             else:
#                                 stock[sym] = stock[sym].append({'tf30':tf30[sym], 'Timestamp':Timestamp[0], 'Date':Date, 'time':tim, 'BidOffer':bidOffer, 'difBO':difBO[sym], 'typeBO':typeBO[sym], 'spread[sym]':spread[sym]}, ignore_index=True)
#                                 sumOrder[sym] = 0
#                                 countOrder[sym] = 0
            
        
# #                                     wr = csv.writer(ref_files[symbols.index(sym)], lineterminator='\n')
# #                                     wr.writerow(forTemp[sym])
# #                                 stock1[sym] = stock[sym].append({'tf30':tf30, 'Timestamp':Timestamp[0], 'Date':Date, 'time':tim, 'BidOffer':bidOffer, 'difBO':difBO[sym], 'typeBO':typeBO[sym], 'spread':spread}, ignore_index=True)
                                


#                                 # sql = "INSERT INTO %s (hour,minute,second,timestamp,bid0,bid1,bid2,bid3,bid4) VALUES ('%s','%s','%s','%s','%s','%s','%s','%s','%s')" % (
#                                 #     sym, HH, mm, SS, Timestamp[0], bidOffer[0], bidOffer[1], bidOffer[2],
#                                 #     bidOffer[3], bidOffer[4])
#                                 #
#                                 # try:
#                                 #     cursor.execute(sql)
#                                 #     # db.commit()
#                                 # except Exception as e:
#                                 #     print e
#                                 #     db.rollback()

#                                 # np.array(forTemp[sym]).dump(open('array.npy', 'wb'))

                                

#                         elif 'ava' in jsonDecoded.keys():
#                             tim = jsonDecoded["tim"]  ##time
#                             HH, mm, SS = getTime(jsonDecoded["tim"])
#                             HH = int(HH)
#                             mm = int(mm)
#                             SS = int(SS)
#                             Timestamp = Datetime2Timestamp(datetime(YYYY, MM, DD, HH, mm, SS))  ###
#                             Timestamp = checkTimestamp(Timestamp,sym)

#                             actVol = jsonDecoded["vol"]  ##vol action
#                             prc = jsonDecoded["prc"]  # Last price action(Baht) must / 100
#                             lastPrice[sym] = prc

#                             # ava = jsonDecoded["ava"]  # Total Trade Value(Baht) must / 100
#                             avo = jsonDecoded["avo"]  # Total Trade Volume(Share)
#                             tradeVol[sym] = avo

#                             bvo = jsonDecoded["bvo"]  # Net Buy Volume(Share)
#                             svo = jsonDecoded["svo"]  # Net Sell Volume(Share)
#                             ovo = jsonDecoded["ovo"]  # Net ATO/ATC Volume(Share)

#                             buyVol[sym] = bvo
#                             sellVol[sym] = svo
#                             auctVol[sym] = ovo

#                             # sid = jsonDecoded["sid"]  ## B = Buy, S = Sell

#                             prr = jsonDecoded["prr"]  # Previous Close must / 100
#                             hgh = jsonDecoded["hgh"]  # high Price must / 100
#                             low = jsonDecoded["low"]  # low Price must / 100
#                             avg = jsonDecoded["avg"]  # Average Price must / 100
#                             prior[sym] = prr
#                             highPrice[sym] = hgh
#                             lowPrice[sym] = low
#                             avgPrice[sym] = avg

#                             sidetransac[sym] = event(jsonDecoded["sid"])

#                             sumOrder[sym] += actVol
#                             countOrder[sym] += 1

#                             if sumOrder[sym] >= 100000:
#                                 count100k[sym] = 1
#                             else:
#                                 count100k[sym] = 0
                                
                                
# t1 = time.time()
# total = t1-t0

# print total

18.3190000057


In [107]:
stock

{'AMANAH':                                                 BidOffer        Date  \
 0      [145, 142, 141, 140, 139, 1700, 176000, 188000...  2017-02-14   
 1      [142, 141, 140, 139, 138, 176000, 188000, 1912...  2017-02-14   
 2      [142, 141, 140, 139, 138, 176000, 188000, 1912...  2017-02-14   
 3      [142, 141, 140, 139, 138, 176000, 188000, 1912...  2017-02-14   
 4      [142, 141, 140, 139, 138, 176000, 188000, 1912...  2017-02-14   
 5      [142, 141, 140, 139, 138, 176000, 188000, 1912...  2017-02-14   
 6      [142, 141, 140, 139, 138, 176000, 188000, 1912...  2017-02-14   
 7      [142, 141, 140, 139, 138, 176000, 188000, 1912...  2017-02-14   
 8      [142, 141, 140, 139, 138, 176000, 188000, 1912...  2017-02-14   
 9      [142, 141, 140, 139, 138, 176000, 188000, 1912...  2017-02-14   
 10     [142, 141, 140, 139, 138, 176000, 188000, 1912...  2017-02-14   
 11     [142, 141, 140, 139, 138, 176000, 188000, 1912...  2017-02-14   
 12     [142, 141, 140, 139, 138, 176000,

In [8]:
ato1['AMANAH'].head()

,Date,Timestamp,marketStatus,op1,op2,pri,time,vol
0,2017-02-14,1.487065e+09,0.0,0.0,0.0,182.0,09:30:21,221800.0
1,2017-02-14,1.487065e+09,0.0,0.0,0.0,182.0,09:30:22,231800.0
2,2017-02-14,1.487065e+09,0.0,0.0,0.0,182.0,09:30:22,241800.0
3,2017-02-14,1.487065e+09,0.0,0.0,0.0,182.0,09:30:22,251800.0
4,2017-02-14,1.487065e+09,0.0,0.0,0.0,182.0,09:30:23,261800.0


In [9]:
ato1['AMANAH'] = ato1['AMANAH'].assign(ln_A = lambda x: np.log(x.vol))
ato1['AMANAH'].head()

,Date,Timestamp,marketStatus,op1,op2,pri,time,vol,ln_A
0,2017-02-14,1.487065e+09,0.0,0.0,0.0,182.0,09:30:21,221800.0,12.309531
1,2017-02-14,1.487065e+09,0.0,0.0,0.0,182.0,09:30:22,231800.0,12.353630
2,2017-02-14,1.487065e+09,0.0,0.0,0.0,182.0,09:30:22,241800.0,12.395866
3,2017-02-14,1.487065e+09,0.0,0.0,0.0,182.0,09:30:22,251800.0,12.436390
4,2017-02-14,1.487065e+09,0.0,0.0,0.0,182.0,09:30:23,261800.0,12.475336


In [42]:
for sym in symbols:
    ato[sym].to_csv("ato\\" +sym+'.csv')
    stock[sym].to_csv("stock\\" +sym+'.csv')
#     dfall[sym].to_csv("stock\\" +sym+'df.csv')

In [13]:
stock['AMANAH']

,BidOffer,Date,Timestamp,difBO,spread,tf30,time,today,tomorrow,typeBO,yesterday
0,"[145, 142, 141, 140, 139, 1700, 176000, 188000...",2017-02-14,1.487066e+09,-13300.0,5.0,0.0,09:56:23,2017-02-14,2017-02-15,Sell,None
1,"[142, 141, 140, 139, 138, 176000, 188000, 1912...",2017-02-14,1.487066e+09,192300.0,2.0,0.0,09:56:13,2017-02-14,2017-02-15,Bid is slot 0,None
2,"[142, 141, 140, 139, 138, 176000, 188000, 1912...",2017-02-14,1.487066e+09,-284400.0,2.0,0.0,09:56:28,2017-02-14,2017-02-15,Offer Not slot 0,None
3,"[142, 141, 140, 139, 138, 176000, 188000, 1912...",2017-02-14,1.487066e+09,100.0,2.0,0.0,09:56:28,2017-02-14,2017-02-15,Offer Not slot 0,None
4,"[142, 141, 140, 139, 138, 176000, 188000, 1912...",2017-02-14,1.487066e+09,-19600.0,2.0,0.0,09:56:29,2017-02-14,2017-02-15,Buy,None
5,"[142, 141, 140, 139, 138, 176000, 188000, 1912...",2017-02-14,1.487066e+09,10000.0,2.0,0.0,09:56:30,2017-02-14,2017-02-15,Offer Not slot 0,None
6,"[142, 141, 140, 139, 138, 176000, 188000, 1912...",2017-02-14,1.487066e+09,3000.0,2.0,0.0,09:56:32,2017-02-14,2017-02-15,Offer is slot 0,None
7,"[142, 141, 140, 139, 138, 176000, 188000, 1912...",2017-02-14,1.487066e+09,40000.0,2.0,0.0,09:56:32,2017-02-14,2017-02-15,Offer is slot 0,None
8,"[142, 141, 140, 139, 138, 176000, 188000, 1912...",2017-02-14,1.487066e+09,100.0,2.0,0.0,09:56:35,2017-02-14,2017-02-15,Offer is slot 0,None
9,"[142, 141, 140, 139, 138, 176000, 188000, 1912...",2017-02-14,1.487066e+09,5000.0,2.0,0.0,09:56:37,2017-02-14,2017-02-15,Bid Not slot 0,None


In [14]:
for sym in symbols:
    for r in range(11):
        print sym, r, stock[sym].loc[stock[sym]['tf30'] == r]['spread'].mean()
    

# stock['OTO'] = stock['OTO'].assign(predict = stock['OTO'].loc[stock['OTO']['tf30'] ==  1]['spread[sym]'])

OTO 0 0.414414414414
OTO 1 2.08527724665
OTO 2 0.258474576271
OTO 3 -1.04522613065
OTO 4 -3.13898305085
OTO 5 -0.252077562327
OTO 6 nan
OTO 7 -0.220394736842
OTO 8 -0.225454545455
OTO 9 -0.942446043165
OTO 10 -2.57074340528
HPT 0 6.83333333333
HPT 1 2.90073529412
HPT 2 -0.324840764331
HPT 3 -0.181818181818
HPT 4 -0.321678321678
HPT 5 -1.61971830986
HPT 6 -1.14285714286
HPT 7 -1.56666666667
HPT 8 -1.15217391304
HPT 9 -0.872093023256
HPT 10 -1.27102803738
AMANAH 0 9.30617977528
AMANAH 1 13.6534750613
AMANAH 2 18.6303615196
AMANAH 3 13.2819407008
AMANAH 4 16.6692851532
AMANAH 5 -7.26968097315
AMANAH 6 -6.25
AMANAH 7 -0.344505494505
AMANAH 8 17.7874587459
AMANAH 9 20.3432364096
AMANAH 10 23.4438344595


In [78]:
# stock[sym].loc[stock[sym]['tf30'] == 6]['spread[sym]']
for r in range(11):
#     stock['AMANAH'] = stock['AMANAH'].assign(ln_A = stock[sym].loc[stock[sym]['tf30'] == r ]['spread[sym]'].mean())
    stock['AMANAH'] = stock['AMANAH'].assign(previous = stock[sym].loc[stock[sym]['tf30'] == r]['spread'].mean())

IndentationError: unexpected indent (<ipython-input-78-efaa6b57fb98>, line 3)

In [48]:
# stock['HPT'].loc[stock['HPT']['tf30'] == 2]['spread'].mean()

dfall['AMANAH']

,BidOffer,Date,Timestamp,difBO,spread,tf30,time,today,tomorrow,typeBO,yesterday,previous,avg,predict
0,"[145, 142, 141, 140, 139, 1700, 176000, 188000...",2017-02-14,1.487066e+09,-13300.0,5.0,0.0,09:56:23,2017-02-14,2017-02-15,Sell,None,NaN,7.0,16.0
1,"[142, 141, 140, 139, 138, 176000, 188000, 1912...",2017-02-14,1.487066e+09,192300.0,2.0,0.0,09:56:13,2017-02-14,2017-02-15,Bid is slot 0,None,NaN,7.0,16.0
2,"[142, 141, 140, 139, 138, 176000, 188000, 1912...",2017-02-14,1.487066e+09,-284400.0,2.0,0.0,09:56:28,2017-02-14,2017-02-15,Offer Not slot 0,None,NaN,7.0,16.0
3,"[142, 141, 140, 139, 138, 176000, 188000, 1912...",2017-02-14,1.487066e+09,100.0,2.0,0.0,09:56:28,2017-02-14,2017-02-15,Offer Not slot 0,None,NaN,7.0,16.0
4,"[142, 141, 140, 139, 138, 176000, 188000, 1912...",2017-02-14,1.487066e+09,-19600.0,2.0,0.0,09:56:29,2017-02-14,2017-02-15,Buy,None,NaN,7.0,16.0
5,"[142, 141, 140, 139, 138, 176000, 188000, 1912...",2017-02-14,1.487066e+09,10000.0,2.0,0.0,09:56:30,2017-02-14,2017-02-15,Offer Not slot 0,None,NaN,7.0,16.0
6,"[142, 141, 140, 139, 138, 176000, 188000, 1912...",2017-02-14,1.487066e+09,3000.0,2.0,0.0,09:56:32,2017-02-14,2017-02-15,Offer is slot 0,None,NaN,7.0,16.0
7,"[142, 141, 140, 139, 138, 176000, 188000, 1912...",2017-02-14,1.487066e+09,40000.0,2.0,0.0,09:56:32,2017-02-14,2017-02-15,Offer is slot 0,None,NaN,7.0,16.0
8,"[142, 141, 140, 139, 138, 176000, 188000, 1912...",2017-02-14,1.487066e+09,100.0,2.0,0.0,09:56:35,2017-02-14,2017-02-15,Offer is slot 0,None,NaN,7.0,16.0
9,"[142, 141, 140, 139, 138, 176000, 188000, 1912...",2017-02-14,1.487066e+09,5000.0,2.0,0.0,09:56:37,2017-02-14,2017-02-15,Bid Not slot 0,None,NaN,7.0,16.0


In [9]:
dfall = pd.DataFrame()

# for row in dfall.itertuples(index=True, name='Pandas'):
#     print getattr(row, "c1"), getattr(row, "c2")


# for n in range(11):
if n == 0:
    # stock[sym].loc[stock[sym]['tf30'] == 1]['spread[sym]'].mean()
    dftest = stock['HPT'].loc[(stock['HPT']['tf30'] == n) & (stock['HPT']['Date'] == today)]                          
    dftest = dftest.assign(previous = stock['HPT'].loc[(stock['HPT']['tf30'] == n+10) & (stock['HPT']['Date'] == yesterday)]['spread'].mean())
    dftest = dftest.assign(avg = stock['HPT'].loc[(stock['HPT']['tf30'] == n)  & (stock['HPT']['Date'] == today)]['spread'].mean())
    dftest = dftest.assign(predict = stock['HPT'].loc[(stock['HPT']['tf30'] == n+1)  & (stock['HPT']['Date'] == today)]['spread'].mean())

    dfall = dfall.append(dftest)


elif n == 10:
    # stock[sym].loc[stock[sym]['tf30'] == 1]['spread[sym]'].mean()
    dftest = stock[sym].loc[(stock['HPT']['tf30'] == n) & (stock[sym]['Date'] == today)]                          
    dftest = dftest.assign(previous = stock['HPT'].loc[(stock['HPT']['tf30'] == n-1) & (stock['HPT']['Date'] == today)]['spread'].mean())
    dftest = dftest.assign(avg = stock['HPT'].loc[(stock['HPT']['tf30'] == n)  & (stock['HPT']['Date'] == today)]['spread'].mean())
    dftest = dftest.assign(predict = stock['HPT'].loc[(stock['HPT']['tf30'] == n-9)  & (stock['HPT']['Date'] == tomorrow)]['spread'].mean())
    dfall = dfall.append(dftest)

else:
    dftest = stock['HPT'].loc[(stock['HPT']['tf30'] == n) & (stock['HPT']['Date'] == today)]                          
    dftest = dftest.assign(previous = stock['HPT'].loc[(stock['HPT']['tf30'] == n-1) & (stock['HPT']['Date'] == today)]['spread'].mean())
    dftest = dftest.assign(avg = stock['HPT'].loc[(stock['HPT']['tf30'] == n)  & (stock['HPT']['Date'] == today)]['spread'].mean())
    dftest = dftest.assign(predict = stock['HPT'].loc[(stock['HPT']['tf30'] == n+1)  & (stock['HPT']['Date'] == today)]['spread'].mean())
    dfall = dfall.append(dftest)

In [12]:
dfall2 = dfall.copy()
dfall3 = dfall2.copy()
dfall4 = dfall2.copy()

for sym in symbols:
    dfall4[sym] = dfall3[sym]['tf30']
    dfall3[sym] = dfall3[sym].drop('Date',axis=1)

In [13]:
# type(dfall4)
dfall4['AMANAH']

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
5         0.0
6         0.0
7         0.0
8         0.0
9         0.0
10        0.0
11        0.0
12        0.0
13        0.0
14        0.0
15        0.0
16        0.0
17        0.0
18        0.0
19        0.0
20        0.0
21        0.0
22        0.0
23        0.0
24        0.0
25        0.0
26        0.0
27        0.0
28        0.0
29        0.0
         ... 
28099    10.0
28100    10.0
28101    10.0
28102    10.0
28103    10.0
28104    10.0
28105    10.0
28106    10.0
28107    10.0
28108    10.0
28109    10.0
28110    10.0
28111    10.0
28112    10.0
28113    10.0
28114    10.0
28115    10.0
28116    10.0
28117    10.0
28118    10.0
28119    10.0
28120    10.0
28121    10.0
28122    10.0
28123    10.0
28124    10.0
28125    10.0
28126    10.0
28127    10.0
28128    10.0
Name: tf30, dtype: float64

In [75]:
dfall4 = pd.get_dummies(dfall4)

In [79]:
result = pd.concat([dfall3, dfall4], axis=1)
result

TypeError: cannot concatenate a non-NDFrame object

In [ ]:
if n == 10:
    # stock[sym].loc[stock[sym]['tf30'] == 1]['spread[sym]'].mean()
    dftest = stock[sym].loc[(stock['HPT']['tf30'] == n) & (stock[sym]['Date'] == today)]                          
    dftest = dftest.assign(previous = stock['HPT'].loc[(stock['HPT']['tf30'] == n-1) & (stock['HPT']['Date'] == today)]['spread'].mean())
    dftest = dftest.assign(avg = stock['HPT'].loc[(stock['HPT']['tf30'] == n)  & (stock['HPT']['Date'] == today)]['spread'].mean())
    dftest = dftest.assign(predict = stock['HPT'].loc[(stock['HPT']['tf30'] == n-9)  & (stock['HPT']['Date'] == tomorrow)]['spread'].mean())
    dfall = dfall.append(dftest)

In [79]:
n=9
if 0<n<10:
    dftest = stock['HPT'].loc[(stock['HPT']['tf30'] == n) & (stock['HPT']['Date'] == today)]                          
    dftest = dftest.assign(previous = stock['HPT'].loc[(stock['HPT']['tf30'] == n-1) & (stock['HPT']['Date'] == today)]['spread'].mean())
    dftest = dftest.assign(avg = stock['HPT'].loc[(stock['HPT']['tf30'] == n)  & (stock['HPT']['Date'] == today)]['spread'].mean())
    dftest = dftest.assign(predict = stock['HPT'].loc[(stock['HPT']['tf30'] == n+1)  & (stock['HPT']['Date'] == today)]['spread'].mean())
    dfall1 = dfall1.append(dftest)

In [14]:
dfall['HPT']

,BidOffer,Date,Timestamp,difBO,spread,tf30,time,today,tomorrow,typeBO,yesterday,previous,avg,predict
0,"[130, 129, 128, 127, 126, 59800, 134100, 26700...",2017-02-14,1.487066e+09,20000.0,130.0,0.0,09:57:03,2017-02-14,2017-02-15,Offer Not slot 0,None,NaN,130.0,6.0
1,"[130, 129, 128, 127, 126, 109800, 134100, 2670...",2017-02-14,1.487066e+09,50000.0,130.0,1.0,10:00:20,2017-02-14,2017-02-15,Bid is slot 0,None,130.0,6.0,-2.0
2,"[130, 129, 128, 127, 126, 109800, 134100, 2672...",2017-02-14,1.487067e+09,200.0,130.0,1.0,10:02:29,2017-02-14,2017-02-15,Bid Not slot 0,None,130.0,6.0,-2.0
3,"[130, 129, 128, 127, 126, 109800, 134100, 2672...",2017-02-14,1.487067e+09,100000.0,130.0,1.0,10:02:40,2017-02-14,2017-02-15,Bid Not slot 0,None,130.0,6.0,-2.0
4,"[130, 129, 128, 127, 126, 109800, 134100, 2672...",2017-02-14,1.487067e+09,10000.0,130.0,1.0,10:02:55,2017-02-14,2017-02-15,Offer Not slot 0,None,130.0,6.0,-2.0
5,"[130, 129, 128, 127, 126, 109800, 134100, 2672...",2017-02-14,1.487067e+09,10000.0,130.0,1.0,10:03:10,2017-02-14,2017-02-15,Offer Not slot 0,None,130.0,6.0,-2.0
6,"[130, 129, 128, 127, 126, 109800, 134100, 2672...",2017-02-14,1.487067e+09,10000.0,130.0,1.0,10:03:20,2017-02-14,2017-02-15,Bid Not slot 0,None,130.0,6.0,-2.0
7,"[130, 129, 128, 127, 126, 109800, 134100, 2672...",2017-02-14,1.487067e+09,10000.0,130.0,1.0,10:03:20,2017-02-14,2017-02-15,Offer Not slot 0,None,130.0,6.0,-2.0
8,"[130, 129, 128, 127, 126, 50800, 134100, 26720...",2017-02-14,1.487067e+09,-59000.0,0.0,1.0,10:03:22,2017-02-14,2017-02-15,Sell,None,130.0,6.0,-2.0
9,"[129, 128, 127, 126, 125, 134100, 267200, 1780...",2017-02-14,1.487067e+09,120200.0,-1.0,1.0,09:56:11,2017-02-14,2017-02-15,Sell,None,130.0,6.0,-2.0


In [9]:
# yesterday = "2017-02-14"
# today = "2017-02-15"
# tomorrow = "2017-02-16"





for sym in symbols:
    dfall[sym] = pd.DataFrame()
    three_tuple = [(current, items[idx - 1] if idx >= 1 else None, items[idx + 1] if idx < len(items) - 1 else None) for idx, current in enumerate(items)]

    for enum, Date in enumerate(items):
        yesterday = three_tuple[enum][1]
        today = three_tuple[enum][0]
        tomorrow = three_tuple[enum][2]

        for n in range(11):
            if n == 0:
                # stock[sym].loc[stock[sym]['tf30'] == 1]['spread[sym]'].mean()
                dftest = stock[sym].loc[(stock[sym]['tf30'] == n) & (stock[sym]['Date'] == today)]
                dftest = dftest.assign(previous =  round(stock[sym].loc[(stock[sym]['tf30'] == n+10) & (stock[sym]['Date'] == yesterday)]['spread'].mean(), 0))
                dftest = dftest.assign(avg =  round(stock[sym].loc[(stock[sym]['tf30'] == n)  & (stock[sym]['Date'] == today)]['spread'].mean(), 0))
                dftest = dftest.assign(predict =  round(stock[sym].loc[(stock[sym]['tf30'] == n+1)  & (stock[sym]['Date'] == today)]['spread'].mean(), 0))

                dfall[sym] = dfall[sym].append(dftest)


            elif n == 10:
                # stock[sym].loc[stock[sym]['tf30'] == 1]['spread[sym]'].mean()
                dftest = stock[sym].loc[(stock[sym]['tf30'] == n) & (stock[sym]['Date'] == today)]
                dftest = stock[sym].loc[(stock[sym]['tf30'] == n) & (stock[sym]['Date'] == today)]
                dftest = dftest.assign(previous = round(stock[sym].loc[(stock[sym]['tf30'] == n-1) & (stock[sym]['Date'] == today)]['spread'].mean(), 0))
                dftest = dftest.assign(avg = round(stock[sym].loc[(stock[sym]['tf30'] == n)  & (stock[sym]['Date'] == today)]['spread'].mean(), 0))
                dftest = dftest.assign(predict = round(stock[sym].loc[(stock[sym]['tf30'] == n-9)  & (stock[sym]['Date'] == tomorrow)]['spread'].mean(), 0))
                dfall[sym] = dfall[sym].append(dftest)

            else:
                dftest = stock[sym].loc[(stock[sym]['tf30'] == n) & (stock[sym]['Date'] == today)]                          
                dftest = dftest.assign(previous = round(stock[sym].loc[(stock[sym]['tf30'] == n-1) & (stock[sym]['Date'] == today)]['spread'].mean(), 0))
                dftest = dftest.assign(avg =  round(stock[sym].loc[(stock[sym]['tf30'] == n)  & (stock[sym]['Date'] == today)]['spread'].mean(), 0))
                dftest = dftest.assign(predict =  round(stock[sym].loc[(stock[sym]['tf30'] == n+1)  & (stock[sym]['Date'] == today)]['spread'].mean(), 0))
                dfall[sym] = dfall[sym].append(dftest)

#FULL 
                    

In [32]:
dfallNA = dfall.copy()
for sym in symbols:
    dfallNA[sym]['predict'].fillna(dfallNA[sym]['avg'], inplace=True)
    dfallNA[sym]['predict'].fillna(dfallNA[sym]['avg'], inplace=True)

    
# fill None Value (Bug from Open Time Randomly.) eg. open before 10:00,  14:30 or No Voltality.

In [41]:
dfall['OTO']

,BidOffer,Date,Timestamp,difBO,spread,tf30,time,today,tomorrow,typeBO,yesterday,previous,avg,predict
0,"[745, 740, 735, 730, 725, 57300, 148400, 13610...",2017-02-14,1.487066e+09,-20000.0,0.0,0.0,09:56:15,2017-02-14,2017-02-15,Buy,None,1.0,1.0,4.0
1,"[745, 740, 735, 730, 725, 57300, 148400, 13610...",2017-02-14,1.487066e+09,100000.0,0.0,0.0,09:56:21,2017-02-14,2017-02-15,Offer Not slot 0,None,1.0,1.0,4.0
2,"[745, 740, 735, 730, 725, 57300, 148400, 13610...",2017-02-14,1.487066e+09,-4000.0,0.0,0.0,09:56:24,2017-02-14,2017-02-15,Buy,None,1.0,1.0,4.0
3,"[745, 740, 735, 730, 725, 57300, 149400, 13610...",2017-02-14,1.487066e+09,1000.0,0.0,0.0,09:56:30,2017-02-14,2017-02-15,Bid Not slot 0,None,1.0,1.0,4.0
4,"[745, 740, 735, 730, 725, 57300, 149400, 13610...",2017-02-14,1.487066e+09,-5000.0,0.0,0.0,09:56:34,2017-02-14,2017-02-15,Offer Not slot 0,None,1.0,1.0,4.0
5,"[745, 740, 735, 730, 725, 107300, 149400, 1361...",2017-02-14,1.487066e+09,50000.0,0.0,0.0,09:56:47,2017-02-14,2017-02-15,Bid is slot 0,None,1.0,1.0,4.0
6,"[745, 740, 735, 730, 725, 207300, 149400, 1361...",2017-02-14,1.487066e+09,100000.0,0.0,0.0,09:56:49,2017-02-14,2017-02-15,Bid is slot 0,None,1.0,1.0,4.0
7,"[750, 745, 740, 735, 730, 45700, 207300, 14940...",2017-02-14,1.487066e+09,-175300.0,1.0,0.0,09:56:53,2017-02-14,2017-02-15,Buy,None,1.0,1.0,4.0
8,"[750, 745, 740, 735, 730, 45700, 207300, 14940...",2017-02-14,1.487066e+09,-1800.0,1.0,0.0,09:56:11,2017-02-14,2017-02-15,Offer is slot 0,None,1.0,1.0,4.0
9,"[750, 745, 740, 735, 730, 31700, 207300, 14940...",2017-02-14,1.487066e+09,-14000.0,1.0,0.0,09:56:59,2017-02-14,2017-02-15,Sell,None,1.0,1.0,4.0


In [33]:
dfall['AMANAH']

,BidOffer,Date,Timestamp,difBO,spread,tf30,time,today,tomorrow,typeBO,yesterday,previous,avg,predict
21370,"[168, 167, 166, 165, 163, 40500, 10000, 18000,...",2017-02-17,1.487325e+09,-90000.0,3.0,0.0,09:56:32,2017-02-17,None,Buy,2017-02-16,-2.582278,12.844920,23.506719
21371,"[169, 168, 167, 166, 165, 56000, 40500, 10000,...",2017-02-17,1.487325e+09,54000.0,4.0,0.0,09:56:36,2017-02-17,None,Buy,2017-02-16,-2.582278,12.844920,23.506719
21372,"[169, 168, 167, 166, 165, 56000, 40500, 10000,...",2017-02-17,1.487325e+09,24700.0,4.0,0.0,09:56:23,2017-02-17,None,Offer is slot 0,2017-02-16,-2.582278,12.844920,23.506719
21373,"[169, 168, 167, 166, 165, 61000, 40500, 10000,...",2017-02-17,1.487325e+09,5000.0,4.0,0.0,09:56:39,2017-02-17,None,Bid is slot 0,2017-02-16,-2.582278,12.844920,23.506719
21374,"[169, 168, 167, 166, 165, 61000, 40500, 10000,...",2017-02-17,1.487325e+09,10000.0,4.0,0.0,09:56:39,2017-02-17,None,Bid Not slot 0,2017-02-16,-2.582278,12.844920,23.506719
21375,"[169, 168, 167, 166, 165, 61000, 40500, 10000,...",2017-02-17,1.487325e+09,-25000.0,4.0,0.0,09:56:47,2017-02-17,None,Buy,2017-02-16,-2.582278,12.844920,23.506719
21376,"[169, 168, 167, 166, 165, 61000, 40500, 10000,...",2017-02-17,1.487325e+09,-15000.0,4.0,0.0,09:56:48,2017-02-17,None,Buy,2017-02-16,-2.582278,12.844920,23.506719
21377,"[169, 168, 167, 166, 165, 61000, 40500, 10000,...",2017-02-17,1.487325e+09,-134000.0,4.0,0.0,09:56:49,2017-02-17,None,Buy,2017-02-16,-2.582278,12.844920,23.506719
21378,"[169, 168, 167, 166, 165, 61000, 40500, 10000,...",2017-02-17,1.487325e+09,-20000.0,4.0,0.0,09:56:51,2017-02-17,None,Buy,2017-02-16,-2.582278,12.844920,23.506719
21379,"[169, 168, 167, 166, 165, 61000, 40500, 10000,...",2017-02-17,1.487325e+09,-20000.0,4.0,0.0,09:56:52,2017-02-17,None,Buy,2017-02-16,-2.582278,12.844920,23.506719


In [188]:
tomorrow = "2017-02-15"
today = "2017-02-14"
n=10
if n == 10:

    # stock[sym].loc[stock[sym]['tf30'] == 1]['spread[sym]'].mean()
    dftest = stock['HPT'].loc[(stock['HPT']['tf30'] == n) & (stock['HPT']['Date'] == today)]                          
    dftest = dftest.assign(previous = stock['HPT'].loc[(stock['HPT']['tf30'] == n-1) & (stock['HPT']['Date'] == today)]['spread'].mean())
    dftest = dftest.assign(avg = stock['HPT'].loc[(stock['HPT']['tf30'] == n)  & (stock['HPT']['Date'] == today)]['spread'].mean())
    dftest = dftest.assign(predict = stock['HPT'].loc[(stock['HPT']['tf30'] == n-9)  & (stock['HPT']['Date'] == tomorrow)]['spread'].mean())

In [32]:
stock['AMANAH']

,BidOffer,Date,Timestamp,difBO,spread,tf30,time,today,tomorrow,typeBO,yesterday
0,"[145, 142, 141, 140, 139, 1700, 176000, 188000...",2017-02-14,1.487066e+09,-13300.0,5.0,0.0,09:56:23,2017-02-14,2017-02-15,Sell,None
1,"[142, 141, 140, 139, 138, 176000, 188000, 1912...",2017-02-14,1.487066e+09,192300.0,2.0,0.0,09:56:13,2017-02-14,2017-02-15,Bid is slot 0,None
2,"[142, 141, 140, 139, 138, 176000, 188000, 1912...",2017-02-14,1.487066e+09,-284400.0,2.0,0.0,09:56:28,2017-02-14,2017-02-15,Offer Not slot 0,None
3,"[142, 141, 140, 139, 138, 176000, 188000, 1912...",2017-02-14,1.487066e+09,100.0,2.0,0.0,09:56:28,2017-02-14,2017-02-15,Offer Not slot 0,None
4,"[142, 141, 140, 139, 138, 176000, 188000, 1912...",2017-02-14,1.487066e+09,-19600.0,2.0,0.0,09:56:29,2017-02-14,2017-02-15,Buy,None
5,"[142, 141, 140, 139, 138, 176000, 188000, 1912...",2017-02-14,1.487066e+09,10000.0,2.0,0.0,09:56:30,2017-02-14,2017-02-15,Offer Not slot 0,None
6,"[142, 141, 140, 139, 138, 176000, 188000, 1912...",2017-02-14,1.487066e+09,3000.0,2.0,0.0,09:56:32,2017-02-14,2017-02-15,Offer is slot 0,None
7,"[142, 141, 140, 139, 138, 176000, 188000, 1912...",2017-02-14,1.487066e+09,40000.0,2.0,0.0,09:56:32,2017-02-14,2017-02-15,Offer is slot 0,None
8,"[142, 141, 140, 139, 138, 176000, 188000, 1912...",2017-02-14,1.487066e+09,100.0,2.0,0.0,09:56:35,2017-02-14,2017-02-15,Offer is slot 0,None
9,"[142, 141, 140, 139, 138, 176000, 188000, 1912...",2017-02-14,1.487066e+09,5000.0,2.0,0.0,09:56:37,2017-02-14,2017-02-15,Bid Not slot 0,None


In [169]:
# stock[sym].loc[stock[sym]['tf30'] == 1]['spread[sym]'].mean()
dftest = stock['HPT'].loc[stock['HPT']['tf30'] == 2]                                
dftest = dftest.assign(previous = stock['HPT'].loc[stock['HPT']['tf30'] == 1]['spread'].mean())
dftest = dftest.assign(avg = stock['HPT'].loc[stock['HPT']['tf30'] == 2]['spread'].mean())
dftest = dftest.assign(predict = stock['HPT'].loc[stock['HPT']['tf30'] == 3]['spread'].mean())

In [126]:
dftest2 = stock['HPT'].loc[stock['HPT']['tf30'] == 3]
previoustest2 = stock[['HPT'].loc[stock[['HPT']['tf30'] == 2]['spread'].mean()
avgtest2 = stock[['HPT'].loc[stock[['HPT']['tf30'] == 3]['spread'].mean()
predicttest2 = stock[['HPT'].loc[stock[['HPT']['tf30'] == 4]['spread'].mean()

dftest2 = dftest2.assign(previous = previoustest2)
dftest2 = dftest2.assign(avg = avgtest2)
dftest2 = dftest2.assign(predict = predicttest2)

In [239]:
yesterday = "2017-02-14"
today = "2017-02-15"
tomorrow = "2017-02-15"

if n==9:
    dftest2 = stock['HPT'].loc[(stock['HPT']['tf30'] == 9) & (stock['HPT']['Date'] == today)]
    previoustest2 = stock[['HPT'].loc[stock[['HPT']['tf30'] == 8]['spread'].mean()
    avgtest2 = stock['HPT'].loc[stock['HPT']['tf30'] == 9]['spread'].mean()
    predict2 = stock['HPT'].loc[(stock['HPT']['tf30'] == 1 ) & (stock['HPT']['Date'] ==tomorrow)]['spread'].mean()

dftest2 = dftest2.assign(previous = round(previoustest2), 0)
dftest2 = dftest2.assign(avg = round(avgtest2), 0)
dftest2 = dftest2.assign(predict = round(predicttest2), 0)

SyntaxError: invalid syntax (<ipython-input-239-cf0f9aec05de>, line 8)

In [141]:
dftest
dftest2

dftest = dftest.append(dftest2)

In [18]:
for n in range(11):
    print round(stock['HPT'].loc[(stock['HPT']['tf30'] == n ) & (stock['HPT']['Date'] == "2017-02-14")]['spread'].mean(),0)

# avgtest2 = stock['HPT'].loc[stock['HPT']['tf30'] == 9]['spread'].mean()

130.0
6.0
-2.0
-1.0
-1.0
-1.0
nan
-1.0
-1.0
-1.0
-2.0


In [19]:
result

NameError: name 'result' is not defined